# Code Handout - Assignment 5 - Ball and Beam

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

## TODO:

Relevant matlab files:

- `BallAndBeamSymbolicTemplate.m` -- symbolic calculations for task 1e, should be completed by the student
- `BallAndBeamODEMatrices.m` -- routine used in the state-space model together with the PD-law, backbone for full model which the student should make
- `BallAndBeamSimulation.m` -- optional, should be completed by the student

Critical

- Make the symbolic template, this is most of the code work
- ???

In [70]:
import sympy as sm
from sympy import vector
from IPython.display import display_latex

# Defining the necessary symbols

J, M, R, g, To = sm.symbols("J M R g T_o")
x, theta = sm.symbols("x theta")
dx, dtheta = sm.symbols(r"dx d\theta")

q = sm.Matrix((x, theta))
dq = sm.Matrix((dx, dtheta))

p = x * sm.Matrix([sm.cos(theta), sm.sin(theta)])\
    + R * sm.Matrix([-sm.sin(theta), sm.cos(theta)])

linear_energy_beam = 0.5 * J * dtheta**2

dp = sm.Matrix.jacobian(p, q)
linear_energy_ball = (0.5 * M * (dq.T @ (dp.T @ dp) @ dq))[0, 0]

I = (2/5) * M * R**2
omega = dtheta + dx / R

rotational_energy_ball = 0.5 * I * omega**2

T = sm.simplify(linear_energy_beam + rotational_energy_ball + linear_energy_ball)

V = M * g * p[1]

Lagrangian = sm.Matrix([T - V])

Lagrangian_q = sm.simplify(sm.Matrix.jacobian(Lagrangian, q))
Lagrangian_qdq = sm.simplify(sm.Matrix.jacobian(Lagrangian_q, dq))
Lagrangian_dq = sm.simplify(sm.Matrix.jacobian(Lagrangian, dq))
Lagrangian_dqdq = sm.simplify(sm.Matrix.jacobian(Lagrangian_dq, dq))

W = Lagrangian_dqdq
RHS = sm.Matrix([0, To]) + sm.simplify(Lagrangian_q.T - Lagrangian_qdq @ dq)

state = sm.Matrix([q, dq])
param = sm.Matrix([J, M, R, g])
